In [1]:
!pip install openai neo4j python-dotenv

import os, dotenv

dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


True

# Generating Cypher queries with GPT-4 on any graph schema

Large language models have a great potential to translate natural language into query language. For example, some people like to use GPT models to translate text to SQL, while others want to use GPT models to construct SPARQL queries. Personally, I prefer exploring how to translate natural language to Cypher query language.
In my experiments, I have noticed there are two approaches to developing an LLM flow that constructs Cypher statements. One option is to provide example queries in the prompt or use the examples to finetune an LLM model. However, the limitation of this approach is that it requires some work to produce the Cypher examples. Therefore, the example Cypher queries must be generated for each graph schema. On the other hand, we can provide an LLM directly with schema information and let it construct Cypher statements based on graph schema information alone. Using the second approach, we could develop a generic Cypher statement model to produce Cypher statements for any input graph schema, as we eliminate the need for any additional work like generating example Cypher statements.
This blog post aims to show you how to implement a Cypher statement-generating model by providing only the graph schema information. We will evaluate the model's Cypher construction capabilities on three graphs with different graph schemas. Currently, the only model I recommend to generate Cypher statements based on only the provided graph schema is GPT-4. Other models like GPT-3.5-turbo or text-davinci-003 aren't that great, and I have yet to find an open-source LLM model that would be good at following instructions in the prompt as well as GPT-4.
## Experiment Setup
I have implemented a Python class that connects to a Neo4j instance and fetches the schema information when initialized. The graph schema information can then be used as input to GPT-4 model.

In [2]:
node_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {labels: nodeLabels, properties: properties} AS output

"""

rel_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {type: nodeLabels, properties: properties} AS output
"""

rel_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE type = "RELATIONSHIP" AND elementType = "node"
RETURN {source: label, relationship: property, target: other} AS output
"""

In [7]:
from neo4j import GraphDatabase
from neo4j.exceptions import CypherSyntaxError
import openai


def schema_text(node_props, rel_props, rels):
    return f"""
  This is the schema representation of the Neo4j database.
  Node properties are the following:
  {node_props}
  Relationship properties are the following:
  {rel_props}
  Relationship point from source to target nodes
  {rels}
  Make sure to respect relationship types and directions
  """


class Neo4jGPTQuery:
    def __init__(self, url, user, password, database, openai_api_key):
        self.driver = GraphDatabase.driver(url, auth=(user, password), database=database)
        openai.api_key = openai_api_key
        # construct schema
        self.schema = self.generate_schema()


    def generate_schema(self):
        node_props = self.query_database(node_properties_query)
        rel_props = self.query_database(rel_properties_query)
        rels = self.query_database(rel_query)
        return schema_text(node_props, rel_props, rels)

    def refresh_schema(self):
        self.schema = self.generate_schema()

    def get_system_message(self):
        return f"""
        Task: Generate Cypher queries to query a Neo4j graph database based on the provided schema definition.
        Instructions:
        Use only the provided relationship types and properties.
        Do not use any other relationship types or properties that are not provided.
        Return only Cypher code and no explanations or apologies.
        Schema:
        {self.schema}
        """

    def query_database(self, neo4j_query, params={}):
        with self.driver.session() as session:
            result = session.run(neo4j_query, params)
            output = [r.values() for r in result]
            output.insert(0, result.keys())
            return output

    def construct_cypher(self, question, history=None):
        messages = [
            {"role": "system", "content": self.get_system_message()},
            {"role": "user", "content": question},
        ]
        # Used for Cypher healing flows
        if history:
            messages.extend(history)

        completions = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0.0,
            max_tokens=1000,
            messages=messages
        )
        return completions.choices[0].message.content

    def run(self, question, history=None, retry=False):
        # Construct Cypher statement
        cypher = self.construct_cypher(question, history)
        print(cypher)
        try:
            return self.query_database(cypher)
        # Self-healing flow
        except CypherSyntaxError as e:
            # If out of retries
            if not retry:
              return "Invalid Cypher syntax"
        # Self-healing Cypher flow by
        # providing specific error to GPT-4
            print("Retrying")
            return self.run(
                question,
                [
                    {"role": "assistant", "content": cypher},
                    {
                        "role": "user",
                        "content": f"""This query returns an error: {str(e)} 
                        Give me a improved query that works without any explanations or apologies""",
                    },
                ],
                retry=False
            )


### I am using my infamous Citation database from Bloom training here

In [8]:
demo_db = Neo4jGPTQuery(
    url="bolt://localhost:7687",
    user="neo4j",
    password="Abcd-1234",
    database="citation",
    openai_api_key=os.environ["OPENAI_API_KEY"],
)


In [5]:
demo_db.run("""
What is the title of the most cited Article ? This report will show only the first value of the first row returned.
""")

MATCH (a:Article)
RETURN a.title
ORDER BY a.n_citation DESC
LIMIT 1;


[['a.title'], ['Seizing Power: Shaders and Storytellers']]

In [9]:
demo_db.run("""
Number of articles per year ? A pie chart expects two fields: a category and a value.
""")

MATCH (a:Article)
RETURN a.year AS category, count(a) AS value


[['category', 'value'],
 [2006, 7536],
 [2009, 817],
 [2003, 3678],
 [2002, 3612],
 [1998, 1620],
 [1999, 1446],
 [2011, 1162],
 [2005, 5056],
 [1994, 209],
 [2000, 1985],
 [2004, 5064],
 [2001, 2719],
 [1993, 213],
 [1997, 902],
 [2013, 959],
 [2012, 1935],
 [2014, 2040],
 [1991, 138],
 [1996, 243],
 [2008, 607],
 [2010, 812],
 [1990, 162],
 [1976, 118],
 [1978, 115],
 [1992, 129],
 [1983, 83],
 [1960, 53],
 [1973, 93],
 [1988, 113],
 [2015, 2075],
 [1979, 77],
 [1972, 116],
 [1995, 228],
 [1985, 94],
 [1977, 83],
 [1974, 91],
 [1980, 49],
 [1959, 42],
 [1981, 85],
 [1975, 80],
 [2007, 640],
 [1982, 81],
 [1989, 198],
 [1962, 201],
 [1969, 142],
 [1987, 81],
 [1961, 166],
 [1984, 90],
 [2016, 2308],
 [1964, 169],
 [1971, 69],
 [1963, 180],
 [1970, 147],
 [1986, 76],
 [1965, 154],
 [1958, 19],
 [1966, 145],
 [1968, 133],
 [1967, 150],
 [2017, 168]]

In [10]:
demo_db.run("""
Provide a good example of a sankey chart using my data model ? A Sankey chart expects Neo4j nodes and weighted relationships.
""")

Here is an example of a Sankey chart using the provided data model:

```
MATCH (a:Author)-[r:CO_AUTHOR]->(b:Author)
WHERE r.collaborations > 5
WITH a, b, r.collaborations AS collaborations
RETURN a.name AS source, b.name AS target, SUM(collaborations) AS value
```

This query will return a set of nodes and weighted relationships between authors who have collaborated on more than 5 articles. The resulting data can be used to create a Sankey chart that shows the flow of collaborations between authors. The "source" and "target" nodes will be the author names, and the "value" will be the number of collaborations between them.


'Invalid Cypher syntax'